# Рекомендация тарифов

В файле предоставлены данные о поведении клиентов, которые уже перешли тарифы телеком-компании. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Порядок исследования

1. Открытие файла с данными и изучение.
2. Разбитие данных на выборки: 25% тестовая выборка, из остатка в пропорции 75:25 - обучающая и валидационная выборка.
3. Подбор оптимальной модели: тестирование параметров дерева реграссии, случайного леса и логистической модели.
4. Применение модели к тестовой выборке.
5. Проверка модели на адекватность.
6. Общие выводы.

## Откройте и изучите файл

In [2]:
#Импорт библиотек для дальнейшей работы
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format 
#в дальнейшей таблице очень много знаков после запятой, ограраничим представление двумя.
pd.set_option('display.max_columns', 50) 
pd.set_option('display.max_rows', 500) 
import matplotlib.pyplot as plt 
import numpy as np
from scipy import stats as st 
from matplotlib import pyplot
import seaborn as sns
from matplotlib import cm
import random

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [4]:
#Импортируем файл с данными 
try:
    data = pd.read_csv('/datasets/users_behavior.csv')
except:
    data = pd.read_csv('users_behavior.csv')
    
    #data.head()
data.index

RangeIndex(start=0, stop=3214, step=1)

В таблице представлены данные о числе звонков, сделанных в месяц, минутах телефонных разговоров, потраченных мб и сообщениях для пользователей тарифов (ультра и смарт). 

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В таблице 3214 записей, типы данных float

In [6]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

Пропусков в данных не обнаружено

In [7]:
data.duplicated().sum()

0

Дублирующихся строк не обнаружено

In [8]:
data.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.00,3214.00,3214.00,3214.00,3214.00
mean,63.04,438.21,38.28,17207.67,0.31
std,33.24,234.57,36.15,7570.97,0.46
min,0.00,0.00,0.00,0.00,0.00
25%,40.00,274.58,9.00,12491.90,0.00
50%,62.00,430.60,30.00,16943.24,0.00
75%,82.00,571.93,57.00,21424.70,1.00
max,244.00,1632.06,224.00,49745.73,1.00


Описание выборки выглядит правдоподобно: звонки, минуты и число сообщений не вызывают ужаса и неприятия. Судя по последнему столбцу, большее половины пользуется тарифом Smart

## Разбейте данные на выборки

Сначала выделим 25% выборки в тестовую. Оставшуюся часть поделим в отношении 75/25 как обучающую и валидационную (тогда в валидационной выборке остается более 600 записей)


In [9]:
from sklearn.model_selection import train_test_split

In [10]:
features = data.drop(['is_ultra'], axis = 1)# извлеките признаки 
target = data['is_ultra']
#Извлекаем тестовую выборку из общей
features_learn, features_test, target_learn, target_test = train_test_split(features, target, test_size=0.25, random_state=12345) # отделите 25% данных для валидационной выборки

#Делим оставшуюся выборку на тренировочную и валидизирующую
features_train, features_valid, target_train, target_valid = train_test_split(features_learn, target_learn, test_size=0.25, random_state=12345) # отделите 25% данных для валидационной выборки




## Исследуйте модели

Поскольку в данном случае у нас стоит задача классификации "тариф ультра или нет", будем оценивать модели по оценке accuracy

Исследуем модель дерева решений

In [9]:
best_model = None
best_result = 0
final_depth = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth) # обучите модель с заданной глубиной дерева
    model.fit(features_train, target_train) # обучите модель
    predictions = model.predict(features_valid) # получите предсказания модели
    result = accuracy_score(target_valid, predictions) # посчитайте качество модели
    if result > best_result:
        best_model = model
        best_result = result
        final_depth = depth
        
display(f'Accuracy лучшей модели решающего дерева:{best_result}, глубина: {final_depth}')

'Accuracy лучшей модели решающего дерева:0.7943615257048093, глубина: 3'

Исследуем модель случайного леса 

In [10]:
best_model = None
best_result = 0
final_depth = 0
estimators = 0
for depth in range(1, 10):
    for est in range(1, 11):
        model = model = RandomForestClassifier(random_state=12345, n_estimators= est, max_depth = depth) # обучите модель с заданной глубиной дерева
        model.fit(features_train, target_train) # обучите модель
        predictions = model.predict(features_valid) # получите предсказания модели
        result = accuracy_score(target_valid, predictions) # посчитайте качество модели
        if result > best_result:
            best_model = model
            best_result = result
            final_depth = depth
            estimators = est
        
display(f'Accuracy лучшей модели случайного леса:{best_result}, глубина: {final_depth}, число деревьев: {estimators}')
    

'Accuracy лучшей модели случайного леса:0.8208955223880597, глубина: 9, число деревьев: 9'

Исследуем модель логистической регрессии

In [21]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
model.predict(features_valid)
print(f'Accuracy лучшей модели логистической регрессии:{model.score(features_valid, target_valid)}')


Accuracy лучшей модели логистической регрессии:0.7412935323383084


Вывод: лучшие показатели из всех моделей дала  модель случайного леса глубиной 9 с числом деревьев 9. Будем использовать ее. 

## Проверьте модель на тестовой выборке

In [12]:
model = RandomForestClassifier(random_state=12345, n_estimators= est, max_depth = depth) # обучите модель с заданной глубиной дерева
model.fit(features_train, target_train) # обучите модель
predictions = model.predict(features_test) # получите предсказания модели
result = accuracy_score(target_test, predictions) # посчитайте качество модели
display(f'Accuracy модели случайного леса по тестовой выборке: {result}')

'Accuracy модели случайного леса по тестовой выборке: 0.7960199004975125'

Вывод: модель случайного леса показала результаты хуже, чем на валидационной выборке, но все же лучше, чем 0,75, которые заданы нам как критерий отбора модели. 

## (бонус) Проверьте модели на адекватность

Проверка модели на адекватность: сравним accuracy выбранной нам модели по тестовой выборке (0,79) с accuracy случайно взятого значения по тестовой выборке.


In [22]:
random_predictions = pd.Series(index=target_test.index)

for i in random_predictions.index:
    random_predictions[i] = random.randint(0, 1)
random_predictions


result = accuracy_score(target_test, random_predictions)
display(f'Accuracy случайно выбранного тарифа по тестовой выборке: {result}')

/tmp/ipykernel_79/324787951.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  random_predictions = pd.Series(index=target_test.index)


'Accuracy случайно выбранного тарифа по тестовой выборке: 0.4937810945273632'

Вывод: все наши модели являются адекватными.

## Общий вывод

Для предсказания, какой тариф следует предложить пользователю, можно выбрать обученную модель типа случайный лес, с 9 estimators, с глубиной 9. На тестовой выборке эта модель показала accuracy: 0.79 (что больше заданного критерия в 0,75). 